1) Importaciones, config y rutas

In [1]:
import os, json, warnings
from pathlib import Path
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import VarianceThreshold

from imblearn.over_sampling import SMOTENC

# --- Config ---
RANDOM_STATE = 42
TARGET = "Exited"
ID_COLS = ["RowNumber", "CustomerId", "Surname"]

# Opciones del preprocesamiento
Q_LOW, Q_HIGH = 0.01, 0.99                # winsorización por cuantiles
TREAT_NUM_PRODUCTS_AS_CAT = True          # mover NumOfProducts a categórica
K_TOP_INIT = 40                           # K inicial; se ajustará dinámicamente

ROOT = Path.cwd().parent
DATA_CANDIDATES = [
    ROOT / "DataBase" / "Churn_Modelling.csv",
    Path.cwd() / "Churn_Modelling.csv",
    ROOT / "Churn_Modelling.csv",
]
DATA_PATH = next((p for p in DATA_CANDIDATES if p.exists()), None)
assert DATA_PATH is not None, "No se encontró Churn_Modelling.csv en ../DataBase/ ni en Code/ ni en la raíz."

OUT_DIR = ROOT / "preproc_datasets"
(OUT_DIR / "full").mkdir(parents=True, exist_ok=True)
(OUT_DIR / "reduced").mkdir(parents=True, exist_ok=True)

print(f"[OK] CWD: {Path.cwd()}")
print(f"[OK] DATA: {DATA_PATH}")
print(f"[OK] OUT : {OUT_DIR}")

[OK] CWD: /Users/luistejada/Downloads/TFE Churn Bancario/Code
[OK] DATA: /Users/luistejada/Downloads/TFE Churn Bancario/DataBase/Churn_Modelling.csv
[OK] OUT : /Users/luistejada/Downloads/TFE Churn Bancario/preproc_datasets


2) Carga, saneo mínimo y descarte de IDs

In [2]:
df = pd.read_csv(DATA_PATH)
assert TARGET in df.columns, f"No existe la columna objetivo '{TARGET}'"

# row_id para poder mapear splits luego
df = df.reset_index(drop=False).rename(columns={"index": "row_id"})

def count_summary(name, y):
    y = pd.Series(y)
    tot = len(y); pos = int((y==1).sum()); neg = int((y==0).sum())
    return {"step": name, "n_total": tot, "n_neg_0": neg, "n_pos_1": pos, "pos_ratio_%": round((pos/tot)*100, 2)}

summaries = [count_summary("raw_full", df[TARGET])]

# Duplicados
before = len(df)
df = df.drop_duplicates()
if len(df) != before:
    summaries.append(count_summary("after_drop_duplicates", df[TARGET]))

# Eliminar IDs no informativos
df = df.drop(columns=ID_COLS)
summaries.append(count_summary("after_drop_ids", df[TARGET]))

pd.DataFrame(summaries)

,step,n_total,n_neg_0,n_pos_1,pos_ratio_%
0,raw_full,10000,7963,2037,20.37
1,after_drop_ids,10000,7963,2037,20.37


3) Split estratificado 60/20/20 (train/val/test)

In [3]:
X = df.drop(columns=[TARGET])
y = df[TARGET].astype(int)

# guardamos los index de row_id y eliminamos la columna
row_id = df["row_id"].copy()
X = X.drop(columns=["row_id"])

X_temp, X_test, y_temp, y_test, rid_temp, rid_test = train_test_split(
    X, y, row_id, test_size=0.20, stratify=y, random_state=RANDOM_STATE
)
X_train, X_val, y_train, y_val, rid_train, rid_val = train_test_split(
    X_temp, y_temp, rid_temp, test_size=0.25, stratify=y_temp, random_state=RANDOM_STATE
)  # 0.25 de 0.8 = 0.20

summaries += [
    count_summary("split_train", y_train),
    count_summary("split_val",   y_val),
    count_summary("split_test",  y_test),
]
pd.DataFrame(summaries)

# Guardar índices de split
split_df = pd.concat([
    pd.DataFrame({"row_id": rid_train, "split": "train"}),
    pd.DataFrame({"row_id": rid_val,   "split": "val"}),
    pd.DataFrame({"row_id": rid_test,  "split": "test"}),
], ignore_index=True)
split_df.to_csv(OUT_DIR / "split_indices.csv", index=False)

# Guardar conteos por step
pd.DataFrame(summaries).to_csv(OUT_DIR / "class_counts_by_step.csv", index=False)

4) Definir columnas num/cat y winsorizar solo con train

In [4]:
NUM_SELECTOR = make_column_selector(dtype_include=np.number)

num_cols = list(NUM_SELECTOR(X_train))
cat_cols = [c for c in X_train.columns if c not in num_cols]

# Mover binarias a categóricas
for b in ["HasCrCard", "IsActiveMember"]:
    if b in num_cols:
        num_cols.remove(b)
    if b not in cat_cols and b in X_train.columns:
        cat_cols.append(b)

# Tratar NumOfProducts como categórica
if TREAT_NUM_PRODUCTS_AS_CAT and "NumOfProducts" in num_cols:
    num_cols.remove("NumOfProducts")
    if "NumOfProducts" not in cat_cols:
        cat_cols.append("NumOfProducts")

# Winsorización por cuantiles (solo numéricas, calculado para train)
q_bounds = {}
for c in num_cols:
    ql, qh = X_train[c].quantile(Q_LOW), X_train[c].quantile(Q_HIGH)
    q_bounds[c] = (float(ql), float(qh))

def clip_with_bounds(df_part, bounds):
    df_part = df_part.copy()
    for c, (ql, qh) in bounds.items():
        df_part[c] = df_part[c].clip(lower=ql, upper=qh)
    return df_part

X_train_num = clip_with_bounds(X_train[num_cols], q_bounds)
X_val_num   = clip_with_bounds(X_val[num_cols],   q_bounds)
X_test_num  = clip_with_bounds(X_test[num_cols],  q_bounds)

# Guardar límites de winsorización
with open(OUT_DIR / "winsor_bounds.json", "w", encoding="utf-8") as f:
    json.dump({"q_low": Q_LOW, "q_high": Q_HIGH, "bounds": q_bounds}, f, ensure_ascii=False, indent=2)

5) OrdinalEncoder para variables categóricas (mapeo estable para SMOTENC) y construir DataFrames encodificados

In [5]:
# OrdinalEncoder para categóricas (mapeo para SMOTENC)
X_train_cat = X_train[cat_cols].astype("category").copy()
X_val_cat   = X_val[cat_cols].astype("category").copy()
X_test_cat  = X_test[cat_cols].astype("category").copy()

oenc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_train_cat_enc = pd.DataFrame(oenc.fit_transform(X_train_cat), columns=cat_cols, index=X_train.index).astype(int)
X_val_cat_enc   = pd.DataFrame(oenc.transform(X_val_cat),   columns=cat_cols, index=X_val.index).astype(int)
X_test_cat_enc  = pd.DataFrame(oenc.transform(X_test_cat),  columns=cat_cols, index=X_test.index).astype(int)

# DF finales encodificados combinando num y cat
X_train_enc = pd.concat([X_train_num, X_train_cat_enc], axis=1)
X_val_enc   = pd.concat([X_val_num,   X_val_cat_enc],   axis=1)
X_test_enc  = pd.concat([X_test_num,  X_test_cat_enc],  axis=1)

# Índices de columnas categóricas para SMOTENC
cat_idx = list(range(len(num_cols), len(num_cols) + len(cat_cols)))

# Guardar categorías del OrdinalEncoder
ord_meta = {
    "ordinal_input_cols": cat_cols,
    "ordinal_categories": {col: [str(x) for x in cats] for col, cats in zip(cat_cols, oenc.categories_)}
}
with open(OUT_DIR / "ordinal_encoder_meta.json", "w", encoding="utf-8") as f:
    json.dump(ord_meta, f, ensure_ascii=False, indent=2)

6) Conjunto Train

In [6]:
def cls_counts(y):
    y = pd.Series(y)
    return {"n": len(y), "neg_0": int((y==0).sum()), "pos_1": int((y==1).sum()), "pos_%": round((y.mean())*100,2)}

print("Train (original):", cls_counts(y_train))

Train (original): {'n': 6000, 'neg_0': 4777, 'pos_1': 1223, 'pos_%': 20.38}


7) RobustScaler (variables numéricas) + OneHotEncoder (cat, drop="if_binary") y transformación

In [8]:
try:
    ohe = OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse_output=False)
except TypeError:
    ohe = OneHotEncoder(drop="if_binary", handle_unknown="ignore", sparse=False)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", RobustScaler(), list(num_cols)),
        ("cat", ohe,            list(cat_cols)),
    ],
    remainder="drop"
)

# Ajustar SIEMPRE con train para evitar sesgos de categorías
preprocessor.fit(X_train_enc)

# Transformaciones sin reducción (FULL)
X_train_full = preprocessor.transform(X_train_enc)
X_val_full   = preprocessor.transform(X_val_enc)
X_test_full  = preprocessor.transform(X_test_enc)

# Nombres de columnas FULL
feature_names = []
if num_cols:
    feature_names += [f"num__{c}" for c in num_cols]
if cat_cols:
    feature_names += list(preprocessor.named_transformers_["cat"].get_feature_names_out(cat_cols))

print("Shapes FULL:", X_train_full.shape, X_val_full.shape, X_test_full.shape)
print("N features:", len(feature_names))

# Metadatos OHE
ohe_meta = {
    "ohe_input_cols": list(cat_cols),
    "ohe_categories": {col: [int(x) if isinstance(x, (np.integer, int)) else str(x)
                             for x in cats]
                       for col, cats in zip(cat_cols, preprocessor.named_transformers_["cat"].categories_)}
}

Shapes FULL: (6000, 15) (2000, 15) (2000, 15)
N features: 15


8) Guardado versión FULL

In [9]:
full_dir = OUT_DIR / "full"
full_dir.mkdir(parents=True, exist_ok=True)

# Matrices FULL
np.save(full_dir / "X_train_full.npy", X_train_full)
np.save(full_dir / "X_val_full.npy",   X_val_full)
np.save(full_dir / "X_test_full.npy",  X_test_full)

# Etiquetas
pd.DataFrame({"Exited": y_train}).to_parquet(full_dir / "y_train.parquet", index=False)
pd.DataFrame({"Exited": y_val}).to_parquet(full_dir / "y_val.parquet",   index=False)
pd.DataFrame({"Exited": y_test}).to_parquet(full_dir / "y_test.parquet",  index=False)

# Nombres de features
pd.DataFrame({"feature": feature_names}).to_parquet(full_dir / "feature_names_full.parquet", index=False)

# Metadatos del preprocesador
preproc_meta = {
    "num_cols": list(num_cols),
    "cat_cols": list(cat_cols),
    "n_features_full": int(len(feature_names)),
    "winsor_bounds_path": str(OUT_DIR / "winsor_bounds.json"),
    "split_indices_path": str(OUT_DIR / "split_indices.csv"),
    "treat_num_products_as_cat": bool(TREAT_NUM_PRODUCTS_AS_CAT),
    "ordinal_encoder_meta_path": str(OUT_DIR / "ordinal_encoder_meta.json"),
    "ohe_meta": ohe_meta,
}
with open(full_dir / "preprocessor_meta.json", "w", encoding="utf-8") as f:
    json.dump(preproc_meta, f, ensure_ascii=False, indent=2)

print("[OK] Exportada versión FULL")

[OK] Exportada versión FULL


9) Chequeo rápido de conteos finales para el reporte

In [10]:
def quick_counts_df():
    return pd.DataFrame([
        {"set": "train", "n": len(y_train), "pos_%": round(y_train.mean()*100,2)},
        {"set": "val",   "n": len(y_val),   "pos_%": round(y_val.mean()*100,2)},
        {"set": "test",  "n": len(y_test),  "pos_%": round(y_test.mean()*100,2)},
    ])

quick_counts_df()

,set,n,pos_%
0,train,6000,20.38
1,val,2000,20.35
2,test,2000,20.35


10 -  χ² por dummy OHE (solo TRAIN) con FDR-BH y guardado

In [2]:
# === χ² sobre dummies OHE ===
import numpy as np, pandas as pd, json, os
from pathlib import Path
from sklearn.feature_selection import chi2

try:
    OUT_DIR 
    full_dir = OUT_DIR / "full"
except NameError:
    ROOT = Path.cwd().parent
    OUT_DIR = ROOT / "preproc_datasets"
    full_dir = OUT_DIR / "full"

assert full_dir.exists(), f"No se encontró {full_dir}. Revisa la ruta."

# Cargar matrices/labels/feature names guardados
X_train_full = np.load(full_dir / "X_train_full.npy")
y_train = pd.read_parquet(full_dir / "y_train.parquet")["Exited"].astype(int)
feature_names = pd.read_parquet(full_dir / "feature_names_full.parquet")["feature"].tolist()

Xtr_full_df = pd.DataFrame(X_train_full, columns=feature_names)

# Dummies OHE = todas las columnas que NO son numéricas escaladas (que empiezan con 'num__')
cat_like = [c for c in Xtr_full_df.columns if not c.startswith("num__")]

# Asegurar binariedad para χ² (0/1). Si alguna no es estrictamente 0/1, la redondeamos.
def is_binary(col):
    vals = pd.unique(Xtr_full_df[col])
    return len(vals) <= 2 and set(np.round(vals, 6)).issubset({0.0, 1.0})

dummy_cols = []
for c in cat_like:
    if not is_binary(c):
        # redondeo defensivo por si hay valores 0/1 en float
        Xtr_full_df[c] = np.round(Xtr_full_df[c]).clip(0,1)
    if is_binary(c):
        dummy_cols.append(c)

Xtr_dum = Xtr_full_df[dummy_cols].astype(float)

# --- Benjamini–Hochberg (FDR) ---
def fdr_bh(pvals, alpha=0.05):
    p = np.asarray(pvals, dtype=float)
    m = p.size
    order = np.argsort(p)
    p_sorted = p[order]
    p_adj_sorted = p_sorted * m / np.arange(1, m+1)
    p_adj_sorted = np.minimum.accumulate(p_adj_sorted[::-1])[::-1]
    p_adj = np.empty_like(p_adj_sorted)
    p_adj[order] = p_adj_sorted
    rej = p_adj <= alpha
    return p_adj, rej

# χ²
chi2_stats, pvals = chi2(Xtr_dum, y_train.to_numpy())
p_adj, rej = fdr_bh(pvals, alpha=0.05)

chi_df = (pd.DataFrame({
    "feature_dummy": dummy_cols,
    "chi2": chi2_stats,
    "p_value": pvals,
    "p_adj_fdr_bh": p_adj,
    "significant_fdr_bh": rej
}))

# Variable original
def orig_var(name):
    return name.split("_", 1)[0] if "_" in name else name

chi_df["orig_feature"] = chi_df["feature_dummy"].map(orig_var)
chi_df = chi_df.sort_values(["p_adj_fdr_bh","feature_dummy"])

# Guardado
out_dir = OUT_DIR / "feature_tests"
out_dir.mkdir(parents=True, exist_ok=True)
chi_path = out_dir / "chi2_train_dummies.csv"
chi_df.to_csv(chi_path, index=False)
print(f"[OK] χ² guardado en {chi_path}")
display(chi_df.head(20))

[OK] χ² guardado en /Users/luistejada/Downloads/TFE Churn Bancario/preproc_datasets/feature_tests/chi2_train_dummies.csv


,feature_dummy,chi2,p_value,p_adj_fdr_bh,significant_fdr_bh,orig_feature
8,NumOfProducts_2,403.369566,1.017255e-89,1.017255e-88,True,NumOfProducts
7,NumOfProducts_1,290.724650,3.456704e-65,1.728352e-64,True,NumOfProducts
9,NumOfProducts_3,152.332788,5.359115e-35,1.786372e-34,True,NumOfProducts
1,Geography_1,136.360222,1.664262e-31,4.160654e-31,True,Geography
6,NumOfProducts_0,104.512356,1.562162e-24,3.124323e-24,True,NumOfProducts
5,IsActiveMember_1,73.791451,8.682181e-18,1.447030e-17,True,IsActiveMember
3,Gender_1,35.695261,2.307245e-09,3.296065e-09,True,Gender
0,Geography_0,29.986503,4.350640e-08,5.438300e-08,True,Geography
2,Geography_2,15.630171,7.701589e-05,8.557321e-05,True,Geography
4,HasCrCard_1,0.760384,3.832085e-01,3.832085e-01,False,HasCrCard


11 - Cramér’s V por variable categórica + resumen por variable

In [3]:
import numpy as np, pandas as pd
from pathlib import Path
from scipy.stats import chi2_contingency


from collections import defaultdict
groups = defaultdict(list)
for col in Xtr_full_df.columns:
    if col.startswith("num__"):
        continue
    base = col.split("_", 1)[0] if "_" in col else col
    groups[base].append(col)

# Cramér’s V corregido
def cramers_v_corrected(contingency_df):
    chi2_stat, _, _, _ = chi2_contingency(contingency_df, correction=False)
    n = contingency_df.to_numpy().sum()
    r, k = contingency_df.shape
    if n <= 1:
        return 0.0
    phi2 = chi2_stat / n
    # corrección de bias
    phi2_corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    r_corr = r - ((r-1)**2)/(n-1)
    k_corr = k - ((k-1)**2)/(n-1)
    denom = min((k_corr-1), (r_corr-1))
    return 0.0 if denom <= 0 else np.sqrt(phi2_corr / denom)

rows = []
reconstructed = {}

for var, cols in groups.items():
    mat = Xtr_full_df[cols].to_numpy()
    if mat.shape[1] > 1:
        idx = mat.argmax(axis=1)
        levels = [c.split("_",1)[1] if "_" in c else c for c in cols]
        labels = pd.Categorical([levels[i] for i in idx], categories=levels)
    else:
        col = cols[0]
        level = col.split("_",1)[1] if "_" in col else "1"
        labels = pd.Categorical(np.where(Xtr_full_df[col] >= 0.5, level, f"not_{level}"))

    reconstructed[var] = pd.Series(labels, index=Xtr_full_df.index, name=var)
    tab = pd.crosstab(reconstructed[var], y_train)
    v = cramers_v_corrected(tab)
    rows.append({"orig_feature": var, "cramers_v": v, "n_levels": tab.shape[0]})

cramer_df = pd.DataFrame(rows).sort_values("cramers_v", ascending=False)

# Resumen por variable combinando χ²
out_dir = OUT_DIR / "feature_tests"
chi_df = pd.read_csv(out_dir / "chi2_train_dummies.csv")

sum_df = (chi_df.groupby("orig_feature", as_index=False)
                .agg(min_q=("p_adj_fdr_bh","min"),
                     n_sig=("significant_fdr_bh","sum"),
                     n_dummies=("feature_dummy","count"))
         ).merge(cramer_df, on="orig_feature", how="left") \
          .sort_values(["min_q","cramers_v"])

# Guardar
cramer_path = out_dir / "cramers_v_train.csv"
summary_path = out_dir / "summary_chi2_per_variable.csv"
cramer_df.to_csv(cramer_path, index=False)
sum_df.to_csv(summary_path, index=False)
print(f"[OK] Cramér’s V -> {cramer_path}")
print(f"[OK] Resumen por variable -> {summary_path}")

display(sum_df.head(15))

[OK] Cramér’s V -> /Users/luistejada/Downloads/TFE Churn Bancario/preproc_datasets/feature_tests/cramers_v_train.csv
[OK] Resumen por variable -> /Users/luistejada/Downloads/TFE Churn Bancario/preproc_datasets/feature_tests/summary_chi2_per_variable.csv


,orig_feature,min_q,n_sig,n_dummies,cramers_v,n_levels
4,NumOfProducts,1.017255e-88,4,4,0.397512,4
1,Geography,4.160654e-31,3,3,0.173208,3
3,IsActiveMember,1.447030e-17,1,1,0.158813,2
0,Gender,3.296065e-09,1,1,0.113752,2
2,HasCrCard,3.832085e-01,0,1,0.016427,2
